In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
   
    PER,
    Doc
)

In [2]:
try:
    df_c = pd.read_csv('data/C.csv', 
                       sep=';', 
                       encoding='cp1251', 
                       index_col=0)
except:
    url = "https://тут_ссылка/С.csv" # ССЫЛКУ НА СКАЧИВАНИЕ СЮДА 
    df_c = pd.read_csv(url) 

df_c.head()

Date                                   UID  \
Number                                                            
1346291   01.01.2023 2:10  e87b2cfa-6159-4aa2-b789-3974a01e0b0a   
1346298   01.01.2023 9:09  d6257b7b-d68c-449f-aef3-6c4af1a2d78f   
1346301  01.01.2023 10:37  927aeffe-0842-4894-8ada-0a91ccdf1692   
1346303  01.01.2023 10:56  9884e4a6-2ce4-485b-913d-8e75c7779302   
1346306  01.01.2023 11:44  4593c91a-8b75-46e4-83e5-557d730abea0   

                                  Desc  Group  Cat  
Number                                              
1346291               Застряли в лифте    NaN  NaN  
1346298      Течь батареи  в спальной     NaN  NaN  
1346301         Течь сверху по стояку     NaN  NaN  
1346303          Течь КНС в 3 подвале     NaN  NaN  
1346306  Течь кровли (забита ливневка)    NaN  NaN

In [3]:
# PEP8
df_c.index.name = df_c.index.name.lower()
df_c.columns = df_c.columns.str.lower()

In [4]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28978 entries, 1346291 to 1425642
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    28978 non-null  object 
 1   uid     28978 non-null  object 
 2   desc    28978 non-null  object 
 3   group   0 non-null      float64
 4   cat     0 non-null      float64
dtypes: float64(2), object(3)
memory usage: 1.3+ MB


**Обработка данных**

**Смысловую нагрузку несёт только desc**

In [5]:
df_c = df_c.drop(columns=['date', 'uid', 'group', 'cat'], axis=0)
df_c

desc
number                                                    
1346291                                   Застряли в лифте
1346298                          Течь батареи  в спальной 
1346301                             Течь сверху по стояку 
1346303                              Течь КНС в 3 подвале 
1346306                      Течь кровли (забита ливневка)
...                                                    ...
1425637  Течь из стыка КНС в подвале, проверить засор в...
1425638                 Нет напора ХВС и совсем пропадает.
1425640                 Обратная тяга вентканала в ванной 
1425641                                             Х/ГВС 
1425642                          Не работает лифт в 1 под.

[28978 rows x 1 columns]

In [6]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

In [7]:
text_c = df_c['desc'].iloc[0]
text_c

'Застряли в лифте'

In [8]:
doc_c = Doc(text_c)

In [9]:
doc_c.segment(segmenter)

In [10]:
doc_c.tag_morph(morph_tagger)
doc_c.parse_syntax(syntax_parser)

In [11]:
for token in doc_c.tokens:
    token.lemmatize(morph_vocab)

In [12]:
h_c = [_.lemma for _ in doc_c.tokens[:10]]

In [13]:
russian_stopwords = stopwords.words('russian');

In [14]:
filtered_words_c = [word for word in h_c if word not in russian_stopwords]
filtered_words = ' '.join(filtered_words_c)

In [15]:
try:
    df_l = pd.read_csv('data/L.csv', 
                       sep=';', 
                       encoding='cp1251', 
                       index_col=0)
except:
    url = "https://тут_ссылка/L.csv" # ССЫЛКУ НА СКАЧИВАНИЕ СЮДА 
    df_l = pd.read_csv(url) 

df_l.head()

Date                                   UID  \
Number                                                                   
1060141         12.03.2021 11:33  7b5dc3bb-1dcb-4954-9f3c-c5dc9c7d45d2   
1112641  2021-05-27 14:16:23.000  8361aa95-b693-444e-a212-8e1da60e8fde   
1112656         31.05.2021 12:07  7ac9fbb2-f72f-471d-9a10-ff191b9be927   
1112657         31.05.2021 12:12  3777d635-b4da-4897-9b89-2ba1aa44d891   
1112668         01.06.2021 11:17  badc0911-6201-4a4e-963e-a63a6a3d408c   

                                        Desc             Group  \
Number                                                           
1060141            Не горит лампа в коридоре  Управление домом   
1112641                 кривой дорожный знак            Дороги   
1112656  Провал колодца на пешеходной дороге       Канализация   
1112657   Низкие температурные параметры ГВС               ГВС   
1112668                    запах канализации       Канализация   

                                                       Cat  
Number                                                      
1060141  Разбитые стекла, сорванные створки оконных пер...  
1112641                                      Общие вопросы  
1112656                                      Общие вопросы  
1112657                                      Общие вопросы  
1112668                                      Общие вопросы

In [16]:
# PEP8
df_l.index.name = df_l.index.name.lower()
df_l.columns = df_l.columns.str.lower()

In [17]:
df_l['group'].unique().__len__()

22

In [18]:
df_l['cat'].unique().__len__()

97

In [19]:
df_l['cat'].unique()

array(['Разбитые стекла, сорванные створки оконных переплетов, форточек, балконных дверных полотен',
       'Общие вопросы', 'Порыв трубы ГВС', 'Общие вопросы ХВС',
       'ГВС (Отсутствие)', 'Межпанельные швы (Протечка)',
       'ГВС (Несоответствие температурных параметров)',
       'Дверные заполнения (входные двери в подъездах)',
       'Неисправности  освещения в общедомовых помещениях',
       'Отлов бродячих животных', 'Техническое обслуживание',
       'ГВС (неудовлетворительное состояние труб в квартире)',
       'ГВС (Порыв трубы)',
       'Короткое замыкание  внутридомовой электрической сети',
       'Водоотведение (Засор)',
       'Отопление (неудовлетворительное состояние труб в квартире)',
       'Замена арматуры с/б', 'Сантехника (замена винтеля)',
       'Проверка канализации', 'ГВС', 'Отопление (Отсутствие)',
       'Центральное отопление', 'Межпанельные швы (Промерзание)',
       'ХВС (Слабое давление)', 'Ликвидация воздушных пробок',
       'Электроэнергия (Отсутстви

In [20]:
subjects_cat = [
    'Разбитые стекла, сорванные створки оконных переплетов, форточек, балконных дверных полотен',
       'Общие вопросы', 'Порыв трубы ГВС', 'Общие вопросы ХВС',
       'ГВС (Отсутствие)', 'Межпанельные швы (Протечка)',
       'ГВС (Несоответствие температурных параметров)',
       'Дверные заполнения (входные двери в подъездах)',
       'Неисправности  освещения в общедомовых помещениях',
       'Отлов бродячих животных', 'Техническое обслуживание',
       'ГВС (неудовлетворительное состояние труб в квартире)',
       'ГВС (Порыв трубы)',
       'Короткое замыкание  внутридомовой электрической сети',
       'Водоотведение (Засор)',
       'Отопление (неудовлетворительное состояние труб в квартире)',
       'Замена арматуры с/б', 'Сантехника (замена винтеля)',
       'Проверка канализации', 'ГВС', 'Отопление (Отсутствие)',
       'Центральное отопление', 'Межпанельные швы (Промерзание)',
       'ХВС (Слабое давление)', 'Ликвидация воздушных пробок',
       'Электроэнергия (Отсутствие)', 'Отключение стояков отопления',
       'Отопление (Нарушение температурного режима)', 'Отопление (Порыв)',
       'Неисправности лифта (Общие проблемы)', 'Канализация',
       'Сантехника', 'Опломбирование приборов ГВС', 'ХВС (Отсутствие)',
       'Неисправности проводки электроснабжения', 'Устранение аварии',
       'Опломбировка ИПУ', 'Монтаж системы домофона в доме',
       'Человек застрял в лифте', 'Кровля (Протечка)',
       'Установка нового домофона', 'Протечки в отдельных местах кровли',
       'Кровля (Ремонт)', 'Приборы учета', 'ХВС (Порыв трубы)',
       'Ремонт дымовых и вентиляционных каналов',
       'Течи в водопроводных кранах и в кранах сливных бачков при унитазах',
       'Отсутствие тяги в дымовых и вентеляционных каналов',
       'Отключение системы электроснабжения',
       'Нарушение связи наружной облицовки', 'Ливнёвки',
       'Техническое обслуживание лифтов', 'Неисправности мусоропроводов',
       'Придомовая территория', 'Повреждения системы водоотвода',
       'Сварочные работы', 'Неисправности электроплиты',
       'Неисправности во вводно-распределительном устройстве',
       'Неплотность в дымоходах и газоходах и сопряжение их с печами',
       'ИПУ', 'ГВС (Слабое давление)', 'Проверка ДВК',
       'Неисправность лифтовой связи', 'Обслуживание диспетчеризации',
       'Техническое обслуживание домофона', 'Консультации по работе МФЦ',
       'Монтаж домофона', 'Подключение к диспетчеризации',
       'Трещины и неисправности в печах, дымоходах и газоходах',
       'Кровля (очистка от снега и наледи)', 'Электроэнергия',
       'Электроэнергия (аварийные работы)', 'Отопление',
       'Канализация (аварийные работы)', 'ХВС',
       'Кровля(очистка от снега и наледи)',
       'Произвести механизирован.уборку терр',
       'Электроэнергия (Замена электрооборудования)',
       'произвести механизированную уборку территории',
       'Общестроительные работы', 'Забит мусоропровод', 'Вывоз мусора',
       'Засор канал', 'Прочие вопросы', 'Отопление (Шум в системе ЦО)',
       'Благоустройство', 'Восстановление связи', 'Газоснабжение',
       'Плановое обслуживание', 'Опломбирование приборов', 'Кровля',
       'ХВС (подмес)', 'Контрольные показания ХВС\\ГВС', 'Дезинфекция',
       'Домофоны', 'ГВС (несоответствие санитарным нормам)',
       'ХВС (несоответствие санитарным нормам)'
]

In [21]:
df_l['group'].unique()

array(['Управление домом', 'Дороги', 'Канализация', 'ГВС',
       'Электроэнергия', 'Газоснабжение', 'Благоустройство', 'ХВС',
       'Общие вопросы', 'Бродячие животные', 'Домофоны', 'Отопление',
       'Сантехника', 'Водоотведение', 'Ливневки', 'Вентиляция',
       'Мусоропровод', 'Уборка  придомовой территории', 'Лифты',
       'Работа МФЦ', 'Диспетчеризация', 'Приборы учета'], dtype=object)

In [22]:
subjects_group = ['Управление домом', 'Дороги', 'Канализация', 'ГВС',
       'Электроэнергия', 'Газоснабжение', 'Благоустройство', 'ХВС',
       'Общие вопросы', 'Бродячие животные', 'Домофоны', 'Отопление',
       'Сантехника', 'Водоотведение', 'Ливневки', 'Вентиляция',
       'Мусоропровод', 'Уборка  придомовой территории', 'Лифты',
       'Работа МФЦ', 'Диспетчеризация', 'Приборы учета'
]

In [23]:
df_l = df_l.drop(columns=['date', 'uid'], axis=1)
df_l

desc             group  \
number                                                                         
1060141                          Не горит лампа в коридоре  Управление домом   
1112641                               кривой дорожный знак            Дороги   
1112656                Провал колодца на пешеходной дороге       Канализация   
1112657                 Низкие температурные параметры ГВС               ГВС   
1112668                                  запах канализации       Канализация   
...                                                    ...               ...   
1346261                    Чуть теплые батареи в квартире.  Управление домом   
1346266    В туалете течет ХВС по трубам с верхнего этажа.  Управление домом   
1346267  Пропало электричество в квартире. Автоматика н...  Управление домом   
1346272           Треснул унитаз, перекрыть воду не могут.  Управление домом   
1346278                                Чуть теплые батареи  Управление домом   

                                                       cat  
number                                                      
1060141  Разбитые стекла, сорванные створки оконных пер...  
1112641                                      Общие вопросы  
1112656                                      Общие вопросы  
1112657                                      Общие вопросы  
1112668                                      Общие вопросы  
...                                                    ...  
1346261        Отопление (Нарушение температурного режима)  
1346266                                         Сантехника  
1346267                                     Электроэнергия  
1346272                                         Сантехника  
1346278        Отопление (Нарушение температурного режима)  

[50990 rows x 3 columns]

In [24]:
text_l = df_l['desc'].iloc[0]
text_l

'Не горит лампа в коридоре'

In [25]:
doc_l = Doc(text_l)

In [26]:
doc_l.segment(segmenter)

In [27]:
doc_l.tag_morph(morph_tagger)
doc_l.parse_syntax(syntax_parser)

In [28]:
for token in doc_l.tokens:
    token.lemmatize(morph_vocab)

In [29]:
h_l = [_.lemma for _ in doc_l.tokens[:10]]

In [30]:
russian_stopwords = stopwords.words('russian');

In [31]:
filtered_words_l = [word for word in h_l if word not in russian_stopwords]
filtered_words = ' '.join(filtered_words_c)

In [32]:
df_l.head(1)

desc             group  \
number                                                 
1060141  Не горит лампа в коридоре  Управление домом   

                                                       cat  
number                                                      
1060141  Разбитые стекла, сорванные створки оконных пер...

In [33]:
df_l.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50990 entries, 1060141 to 1346278
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   desc    50988 non-null  object
 1   group   50990 non-null  object
 2   cat     50990 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


In [34]:
df_l = df_l.dropna()

In [35]:
from nltk.corpus import wordnet

In [36]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KaLbIch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [37]:
import re

In [38]:
# лемматизируем
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


lemmatizer = WordNetLemmatizer()

df_l['desc'] = df_l['desc'].apply(lambda text : 
                             " ".join(
                                 re.sub(r'[^а-яА-Я]', ' ',
                                        ''.join(lemmatizer.lemmatize(text.lower(), 
                                                                     get_wordnet_pos(text.lower())))).split()))

df_l.head()

C:\Users\KaLbIch\AppData\Local\Temp\ipykernel_6528\886329639.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l['desc'] = df_l['desc'].apply(lambda text :


desc             group  \
number                                                           
1060141            не горит лампа в коридоре  Управление домом   
1112641                 кривой дорожный знак            Дороги   
1112656  провал колодца на пешеходной дороге       Канализация   
1112657   низкие температурные параметры гвс               ГВС   
1112668                    запах канализации       Канализация   

                                                       cat  
number                                                      
1060141  Разбитые стекла, сорванные створки оконных пер...  
1112641                                      Общие вопросы  
1112656                                      Общие вопросы  
1112657                                      Общие вопросы  
1112668                                      Общие вопросы

In [39]:
# лемматизируем
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


lemmatizer = WordNetLemmatizer()

df_l['group'] = df_l['group'].apply(lambda text : 
                             " ".join(
                                 re.sub(r'[^а-яА-Я]', ' ',
                                        ''.join(lemmatizer.lemmatize(text.lower(), 
                                                                     get_wordnet_pos(text.lower())))).split()))

df_l.head()

C:\Users\KaLbIch\AppData\Local\Temp\ipykernel_6528\1646116935.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l['group'] = df_l['group'].apply(lambda text :


desc             group  \
number                                                           
1060141            не горит лампа в коридоре  управление домом   
1112641                 кривой дорожный знак            дороги   
1112656  провал колодца на пешеходной дороге       канализация   
1112657   низкие температурные параметры гвс               гвс   
1112668                    запах канализации       канализация   

                                                       cat  
number                                                      
1060141  Разбитые стекла, сорванные створки оконных пер...  
1112641                                      Общие вопросы  
1112656                                      Общие вопросы  
1112657                                      Общие вопросы  
1112668                                      Общие вопросы

In [40]:
# лемматизируем
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


lemmatizer = WordNetLemmatizer()

df_l['cat'] = df_l['cat'].apply(lambda text : 
                             " ".join(
                                 re.sub(r'[^а-яА-Я]', ' ',
                                        ''.join(lemmatizer.lemmatize(text.lower(), 
                                                                     get_wordnet_pos(text.lower())))).split()))

df_l.head()

C:\Users\KaLbIch\AppData\Local\Temp\ipykernel_6528\2793607086.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l['cat'] = df_l['cat'].apply(lambda text :


desc             group  \
number                                                           
1060141            не горит лампа в коридоре  управление домом   
1112641                 кривой дорожный знак            дороги   
1112656  провал колодца на пешеходной дороге       канализация   
1112657   низкие температурные параметры гвс               гвс   
1112668                    запах канализации       канализация   

                                                       cat  
number                                                      
1060141  разбитые стекла сорванные створки оконных пере...  
1112641                                      общие вопросы  
1112656                                      общие вопросы  
1112657                                      общие вопросы  
1112668                                      общие вопросы

In [41]:
# лемматизируем
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


lemmatizer = WordNetLemmatizer()

df_c['desc'] = df_c['desc'].apply(lambda text : 
                             " ".join(
                                 re.sub(r'[^а-яА-Я]', ' ',
                                        ''.join(lemmatizer.lemmatize(text.lower(), 
                                                                     get_wordnet_pos(text.lower())))).split()))

df_c.head()

desc
number                              
1346291             застряли в лифте
1346298      течь батареи в спальной
1346301        течь сверху по стояку
1346303           течь кнс в подвале
1346306  течь кровли забита ливневка

In [58]:
# выделяем признаки и целевую переменную
X = df_l.drop(['group', 'cat'], axis=1)
y = df_l['group']

In [59]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [73]:
from nltk.corpus import stopwords
stopwords = stopwords.words("russian")

In [77]:
corpus_X = X['desc'].to_numpy().tolist()
corpus_y = y.to_numpy().tolist()

In [79]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
vector_X = vectorizer.fit_transform(corpus_X)
vector_y = vectorizer.fit_transform(corpus_y)

In [80]:
df_l.head(1)

desc             group  \
number                                                 
1060141  не горит лампа в коридоре  управление домом   

                                                       cat  
number                                                      
1060141  разбитые стекла сорванные створки оконных пере...

In [82]:
df_ = df_l.drop(['desc', 'group', 'cat'], axis=1)

In [83]:
df_ = pd.concat([df_, pd.DataFrame(vector_X.toarray(), 
                                   columns=[f'desc_{col}' for col in range(vector_X.shape[1])])], sort=False, axis=1)


In [84]:
df_ = pd.concat([df_, pd.DataFrame(vector_y.toarray(), 
                                   columns=[f'group_{col}' for col in range(vector_y.shape[1])])], sort=False, axis=1)

In [88]:
from sklearn.cluster import KMeans

In [89]:
kmeans = KMeans(random_state=0).fit(df_)

MemoryError: Unable to allocate 6.80 GiB for an array with shape (101976, 8953) and data type float64

In [ ]:
kmeans.labels_

In [ ]:
df_['cluster'] = kmeans.labels_

In [ ]:
df_.head()